In [ ]:
# MIT License
#
# Copyright (C) The Adversarial Robustness Toolbox (ART) Authors 2021
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated
# documentation files (the "Software"), to deal in the Software without restriction, including without limitation the
# rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit
# persons to whom the Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of the
# Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
# WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
# TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
"""
This module defines a base class for EoT in PyTorch.
"""
from abc import abstractmethod
import logging
from typing import Optional, Tuple, TYPE_CHECKING, List

from art.preprocessing.preprocessing import PreprocessorPyTorch

if TYPE_CHECKING:
    import torch

logger = logging.getLogger(__name__)


class EoTPyTorch(PreprocessorPyTorch):
    """
    This module defines a base class for EoT in PyTorch.
    """

    def __init__(
        self,
        nb_samples: int,
        clip_values: Tuple[float, float],
        apply_fit: bool = False,
        apply_predict: bool = True,
    ) -> None:
        """
        Create an instance of EoTPyTorch.

        :param nb_samples: Number of random samples per input sample.
        :param clip_values: Tuple of float representing minimum and maximum values of input `(min, max)`.
        :param apply_fit: True if applied during fitting/training.
        :param apply_predict: True if applied during predicting.
        """
        super().__init__(is_fitted=True, apply_fit=apply_fit, apply_predict=apply_predict)

        self.nb_samples = nb_samples
        self.clip_values = clip_values
        EoTPyTorch._check_params(self)

    @abstractmethod
    def _transform(
        self, x: "torch.Tensor", y: Optional["torch.Tensor"], **kwargs
    ) -> Tuple["torch.Tensor", Optional["torch.Tensor"]]:
        """
        Internal method implementing the transformation per input sample.

        :param x: Input samples.
        :param y: Label of the samples `x`.
        :return: Transformed samples and labels.
        """
        "The transform will change the input to make it more adversarially robust."
        import cv2
        #tensor_numpy= x.cpu().numpy() # make sure tensor is on cpu
        tensor_to_numpy = x.numpy()   
        cv2.imwrite(tensor_to_numpy, "the_clone.jpg")
        # edit the image
        from matplotlib import image
        from matplotlib import pyplot as plt

        # to read the image stored in the working directory
        data = image.imread('the_clone.jpg')
        #parameters of height and width
        width = data.shape[2] 
        height = data.shape[1]
        x1 = random.randint(0, width)
        x2 = random.randint(x1, width)
        y1 = random.randint(0, width)
        y2 = random.randint(y1, width)
        x_line = [x1, x2]
        y_line = [y1, y2]
        plt.plot(x_line, y_line, color="white", linewidth=3)
        plt.axis('off')
        #now convert the figure to jpeg, then to a tensor
        plt.savefig('the_clone.jpg')
        from torchvision.io.ImageReadMode import read_image
        tensor = read_image("the_clone.jpg") #by default is ImageReadMode.Unchanged
        return torch.clamp(tensor, min=self.clip_values[0], max=self.clip_values[1]), y

    def forward(
        self, x: "torch.Tensor", y: Optional["torch.Tensor"] = None
    ) -> Tuple["torch.Tensor", Optional["torch.Tensor"]]:
        """
        Apply transformations to inputs `x` and labels `y`.

        :param x: Input samples.
        :param y: Label of the samples `x`.
        :return: Transformed samples and labels.
        """
        import torch  # lgtm [py/repeated-import]

        x_preprocess_list = list()
        y_preprocess_list: List["torch.Tensor"] = list()

        for i_image in range(x.shape[0]):
            for _ in range(self.nb_samples):
                x_i = x[i_image]
                y_i: Optional["torch.Tensor"]
                if y is not None:
                    y_i = y[i_image]
                else:
                    y_i = None
                x_preprocess, y_preprocess_i = self._transform(x_i, y_i)
                x_preprocess_list.append(x_preprocess)

                if y is not None and y_preprocess_i is not None:
                    y_preprocess_list.append(y_preprocess_i)

        x_preprocess = torch.stack(x_preprocess_list, dim=0)
        if y is None:
            y_preprocess = y
        else:
            y_preprocess = torch.stack(y_preprocess_list, dim=0)

        return x_preprocess, y_preprocess

    def _check_params(self) -> None:

        if not isinstance(self.nb_samples, int) or self.nb_samples < 1:
            raise ValueError("The number of samples needs to be an integer greater than or equal to 1.")

        if not isinstance(self.clip_values, tuple) or (
            len(self.clip_values) != 2
            or not isinstance(self.clip_values[0], (int, float))
            or not isinstance(self.clip_values[1], (int, float))
            or self.clip_values[0] > self.clip_values[1]
        ):
            raise ValueError("The argument `clip_Values` has to be a float or tuple of two float values as (min, max).")
            